In [1]:
import pandas as pd

In [2]:
df1 = pd.read_stata("1314_sec_2ab.dta", convert_categoricals=False)

In [3]:
df1.sample(10)

,hhcode,psu,province,region,idc,s2aq01,s2aq02,s2aq03,s2bq01,s2bq02,...,s2bq19d,s2bq19e,s2bq19f,s2bq19g,s2bq19h,s2bq19i,filter__,hhcode_new,stratum,psu_new
53516,2.724100e+09,27241001,2,1,2,2.0,NaN,1.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2.721040e+09,27210,27210401
69131,3.211210e+09,32112101,3,2,52,NaN,NaN,NaN,1,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3.212110e+09,32121,32121101
33805,2.333200e+09,23332003,2,2,52,NaN,NaN,NaN,3,NaN,...,700.0,0.0,0.0,0.0,800.0,13000.0,1,2.302030e+09,23020,23020303
76572,3.312100e+09,33121002,3,1,1,2.0,NaN,1.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3.311020e+09,33110,33110202
3081,1.142200e+09,11422002,1,2,2,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1.102020e+09,11020,11020202
59742,2.912100e+09,29121002,2,1,56,NaN,NaN,NaN,1,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2.911020e+09,29110,29110202
51764,2.713100e+09,27131003,2,1,53,NaN,NaN,NaN,3,NaN,...,0.0,0.0,0.0,0.0,200.0,1500.0,0,2.711030e+09,27110,27110303
69996,3.214100e+09,32141001,3,1,8,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3.211040e+09,32110,32110401
79983,3.351100e+09,33511001,3,1,51,2.0,NaN,1.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3.351010e+09,33510,33510101
71449,3.223200e+09,32232001,3,2,5,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3.202030e+09,32020,32020301


In [4]:
# convert hhcode column to string for easy access
df1["hhcode"] = df1["hhcode"].astype("str")

# cleaning important columns
df1["ever_admitted"] = "no"
df1.loc[(df1["s2bq01"] == 1 | 2), "ever_admitted"] = "yes"

df1["currently_enrolled"] = "no"
df1.loc[(df1["s2bq01"] == 3), "currently_enrolled"] = "yes"

In [5]:
# Reducing to the questions of interest:
df1_rel = df1[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        'stratum'
    ]
].copy()

In [6]:
df10 = pd.read_stata("1314_roster.dta")

In [7]:
df10["hhcode"] = df10["hhcode"].astype("str")

df10["sex"] = df10["s1aq04"]
df10["marital_status"] = df10["s1aq07"]

In [8]:
df10_rel = df10[["hhcode", "age", "idc", "sex", "marital_status"]].copy()

In [9]:
years13_14 = pd.merge(df1_rel, df10_rel, on=["hhcode", "idc"], how='inner', indicator=False)

In [10]:
years13_14.loc[(years13_14["region"] == 1), "region"] = "urban"
years13_14.loc[(years13_14["region"] == 2), "region"] = "rural"

# filtering the age for individuals between the ages of 4 - 15
years13_14 = years13_14.loc[(years13_14["age"] > 4) & (years13_14["age"] <= 15)]

years13_14.loc[(years13_14["region"] == 1), "region"] = "urban"
years13_14.loc[(years13_14["region"] == 2), "region"] = "rural"

years13_14["subprovince code"] = years13_14["hhcode"].apply(lambda x: x[0:4])
years13_14["subprovince code"] = years13_14["subprovince code"].astype("int")

#years13_14["province"] = years13_14["province"].astype("int")

years13_14["province"] = years13_14["province"].map(
    {1: "K.P.K", 2: "Punjab", 3: "Sindh", 4: "Balochistan"}
)

# modifying marital status to align with desired output
years13_14["marital_status"].replace(
    [
        "unmarried/never married",
        "currently married",
        "widow",
        " divorced",
        "nikkah has been solemnised but the rukhsati has not taken place code",
    ],
    [1, 2, 3, 4, 5],
    inplace=True,
)

# creating year column with lowest year as value
years13_14["year"] = 2013

years13_14.rename(columns = {'stratum':'code'}, inplace = True)

In [11]:
strat = pd.read_csv('1314_stratum.csv', usecols=['stratum', 'code'])

In [13]:
years13_14 = pd.merge(years13_14, strat, on=['code'], how='inner', indicator=False)

In [19]:
years13_14 = years13_14.drop('code', 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [21]:
years13_14.to_csv("years13_14_f.csv")